In [9]:
'''Changing the txt file to csv file'''

# import csv

# txt_file = r"/Users/liyatilahun/Downloads/booksummaries/booksummaries.txt"
# csv_file = r"book_detail.csv"

# with open(txt_file, "r") as in_text:
#     in_reader = csv.reader(in_text, delimiter = '\t')
#     with open(csv_file, "w") as out_csv:
#         out_writer = csv.writer(out_csv)
#         for row in in_reader:
#             out_writer.writerow(row)

In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer #histogram(array) representation of words
from sklearn.metrics.pairwise import cosine_similarity #to calculate the cosine_similarity
from sklearn import datasets
from scipy import sparse

#data from goodbooks-10k dataset
df = pd.read_csv('book_detail.csv')
df.drop(df.columns[[-1,-6,-7]], axis=1, inplace=True) #delete the last column

def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
        
display_all(df.head())
# print(df.keys())





,title,author,year,genre
0,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":..."
1,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N..."
2,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi..."
3,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN
4,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90..."


In [12]:
df['genre']

0        {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt":...
1        {"/m/06n90": "Science Fiction", "/m/0l67h": "N...
2        {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...
3                                                      NaN
4        {"/m/03lrw": "Hard science fiction", "/m/06n90...
                               ...                        
16554                                                  NaN
16555     {"/m/01jfsb": "Thriller", "/m/02xlf": "Fiction"}
16556                         {"/m/0xdf": "Autobiography"}
16557                                                  NaN
16558    {"/m/02ql9": "Epistolary novel", "/m/014dfn": ...
Name: genre, Length: 16559, dtype: object